In [1]:
import os
import pickle
JAVA_SO_POSTS_STORE_PATH = 'C:/workspace/SOworkspace/data/so_posts/java/'
post_file_names = os.listdir(JAVA_SO_POSTS_STORE_PATH)
post_file_names[0:1]

['posts_0.pkl']

In [9]:
def get_general_api_name(api_name : str):
    '''
    不同文档间的api路径可能不一样，但是后面三个的路径应该是一样的，做一个粗略的api消歧
    '''
    ret = ''
    try:
        tokens = api_name.split('/')
        general_tokens = []
        recording = False
        for token in tokens:
            if token == 'java' or token == 'javax':
                recording = True
            if recording:
                general_tokens.append(token)
        ret = '/'.join(general_tokens)
        #消除特殊字符
        ret = ret.replace('%3C', '<')
        ret = ret.replace('%3E', '>')
        ret = ret.replace('%5B', '[')
        ret = ret.replace('%5D', ']')
    except:
        ret = 'nonsence'
    return ret

get_general_api_name('file:///C:/workspace/SOworkspace/apidocs/javadocs/api/java.base/java/lang/Boolean.html#equals(java.lang.Object)')

'java/lang/Boolean.html#equals(java.lang.Object)'

In [4]:
import networkx as nx
JAVADOC_CP_PATH = 'C:/workspace/SOworkspace/backup/concept_map_javadoc20200801.gexf'
concept_map = nx.read_gexf(JAVADOC_CP_PATH)

In [5]:
nodes = list(concept_map.nodes)
len(nodes)

52232

In [6]:
nodes = [get_general_api_name(node) for node in nodes if get_general_api_name(node) != '']
len(nodes)

45417

In [10]:
concept_post_map = {}
concepts = set(nodes)
for concept in nodes:
    concept_post_map[concept] = []
for post_file_name in post_file_names:
    with open(os.path.join(JAVA_SO_POSTS_STORE_PATH, post_file_name), 'rb') as rf:
        temp_posts = pickle.load(rf)
        for post in temp_posts:
            links = [get_general_api_name(link) for link in post['Links']]
            item = {
                'Id' : post['Id'],
                'Title' : post['Title'],
                'Links' : post['Links']
            }
            for link in links:
                if link in concepts:
                    concept_post_map[link].append(item)


TypeError: file must have a 'write' attribute

In [14]:
with open('C:/workspace/SOworkspace/data/cache/concept_post_map_direct_initial.pkl', 'wb') as wf:
    pickle.dump(concept_post_map, wf)